In [1]:
import os
import pandas as pd

df = pd.read_csv('data/339_output_ortho/339_ncbi_b10_2.csv')

df.head()

,guide_pam,guide_score,guide_strand,guide_sequence,guide_endonuclease,guide_genomic_location,guide_sequence_with_context,cds_name,protein_id,cds_string_id,cds_locus_tag,cds_integer_id,cds_ref_species,cds_species_name,cds_orthologous_to,cds_orthologous_to_ref_prot_id
0,GG,56.07,F,AACTTCGGCGACATCCTGGC,cas9,826,NaN,THR4,XP_041547107.1,XP_041547107.1,AKAW2_70223S,15,kluyveromyces_marxianus,aspergillus_luchuensis,THR4,XP_022675622.1
1,GG,50.67,F,AACTTCGGCGACATCCTGGC,cas9,925,NaN,THR4,XP_024320180.1,XP_024320180.1,VC83_08619,17,kluyveromyces_marxianus,pseudogymnoascus_destructans,THR4,XP_022675622.1
2,GG,56.28,F,AACTTCGGCGACATCCTGGC,cas9,826,NaN,NaN,XP_025542779.1,XP_025542779.1,BO79DRAFT_207634,7,kluyveromyces_marxianus,aspergillus_costaricensis,THR4,XP_022675622.1
3,GG,56.07,F,AACTTCGGCGACATCCTGGC,cas9,826,NaN,NaN,XP_025512183.1,XP_025512183.1,BO85DRAFT_428187,15,kluyveromyces_marxianus,aspergillus_piperis,THR4,XP_022675622.1
4,GG,56.28,F,AACTTCGGCGACATCCTGGC,cas9,826,NaN,NaN,XP_025484599.1,XP_025484599.1,BO87DRAFT_372335,1,kluyveromyces_marxianus,aspergillus_neoniger,THR4,XP_022675622.1


In [2]:
df.guide_sequence.unique().shape

(50,)

In [4]:
df.shape

(711, 16)

In [4]:
print(len(df.cds_orthologous_to.unique()))
df.cds_orthologous_to.unique()

17


array(['THR4', 'LEU1', 'LYS9', 'ADE6', 'LYS2', 'TRP5', 'ADE1', 'LYS4',
       'ARG1', 'ILV3', 'LYS1', 'HIS7', 'ARG4', 'MET2', 'ARG3', 'ADE13',
       'HIS4'], dtype=object)

In [5]:
df.cds_species_name.value_counts()

colletotrichum_gloeosporioides    7
colletotrichum_karsti             7
alternaria_burnsii                6
dothidotthia_symphoricarpi        5
candida_subhashii                 5
                                 ..
schizosaccharomyces_octosporus    1
trichoderma_harzianum             1
kwoniella_dejecticola             1
schizosaccharomyces_japonicus     1
trichoderma_reesei                1
Name: cds_species_name, Length: 339, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

inputs_species = pd.read_csv('data/input/final_standard_ncbi_input_species.csv')

train, test = train_test_split(inputs_species, test_size=0.1)

In [7]:
train.to_csv('data/input/train_ncbi_species.csv', index=False)
test.to_csv('data/input/test_ncbi_species.csv', index=False)

In [8]:
# run allegro with 305_ncbi_species.csv

os.system(
    ("conda run -n allegro python src/main.py " +
    "--experiment_name train_test "
    "--input_species_path data/input/train_ncbi_species.csv " +
    "--output_directory data/train_test_output/ " +
    "--mode from_orthogroups " +
    "--objective min " +
    "--scorer dummy "
    )
)

In [20]:
guides_to_hit_305 = pd.read_csv('../data/339_output_ortho/305_ncbi_b0.csv')

guides_to_hit_305.cds_orthologous_to.unique()

array(['ADE6', 'ARG3', 'LYS4', 'ADE13', 'ARG1', 'MET2', 'LEU1', 'TRP5',
       'ADE1', 'LYS1', 'LYS9', 'LYS2', 'THR4', 'HIS7'], dtype=object)

In [21]:
guides_to_hit_305.cds_orthologous_to.unique().shape

(14,)

In [22]:
guides_to_hit_305.guide_sequence.unique().shape

(47,)

In [54]:
# Let's hit the genomes of the other 34 species with these 47 guides.
# Then we see where the hits occur.
# Then, we see if the hits are in a coding region and if they are hitting the right genes

In [11]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

# species_name: full_genome
full_genomes: dict[str, str] = dict()

for idx, row in test.iterrows():
    full_genomes[row.species_name] = ''
    scaffolds = list(SeqIO.parse(open('../data/input/genomes/' + row.genome_file_name), 'fasta'))

    for s in scaffolds:
        full_genomes[row.species_name] += str(s.seq)


In [57]:
# sequence: (species_name, strand, [locations])

In [10]:
from src.utils.find_cas9_guides_in_seq import GuideFinder


class NameStrandLoc:
    def __init__(self) -> None:
        self.strands: list[str] = list()
        self.species: list[str] = list()
        self.genomic_locs: list[int] = list()


    @property
    def attributes(self) -> dict[str, list[str] | list[int]]:
        return dict({
            'strands': self.strands,
            'species': self.species,
            'genomic_locs': self.genomic_locs,
        })


    def update(self, species: str, strand: str, loc: int) -> None:
        self.strands.append(strand)
        self.species.append(species)
        self.genomic_locs.append(loc)


# gf = GuideFinder()
# seq_to_info: dict[str, NameStrandLoc] = dict()


# for key, value in full_genomes.items():
#     list_of_tupes = gf.find_guides_and_indicate_strand(value)

#     for t in list_of_tupes:
#         seq_to_info.setdefault(t[0], NameStrandLoc()).update(species=key, strand=t[1], loc=t[2])

In [11]:
import pickle

# pickle.dump(seq_to_info, open('seq_to_info.pickle', 'wb'))
# with open('seq_to_info.pickle', 'wb') as f:
#     pickle.dump(seq_to_info, f)
seq_to_info = pickle.load(open('seq_to_info.pickle', 'rb'))

In [12]:
guide_targets_in_34 = list()
for seq in guides_to_hit_305.guide_sequence.unique():
    if seq in seq_to_info:
        guide_targets_in_34.append(seq)

species_targets = list()
target_seq_objects = dict()
for target in guide_targets_in_34:
    species_targets.extend(seq_to_info[target].species)
    target_seq_objects[target] = seq_to_info[target]


print('Unique species hit by 305 guides:')
print(set(species_targets))
print('Total #:', len(set(species_targets)), '/', len(test))

Unique species hit by 305 guides:
{'candida_pseudohaemulonii', 'colletotrichum_karsti', 'candida_pseudojiufengensis', 'naumovozyma_dairenensis', 'penicillium_digitatum', 'lodderomyces_elongisporus', 'microsporum_canis', 'daldinia_loculata', 'pseudogymnoascus_destructans', 'aspergillus_ibericus', 'daldinia_childiae', 'dothidotthia_symphoricarpi', 'nannizzia_gypsea', 'serpula_lacrymans', 'aspergillus_fischeri', 'neurospora_crassa', 'diplodia_corticola', 'trametes_versicolor', 'cladophialophora_carrionii', 'neoarthrinium_moseri', 'fusarium_oxysporum', 'penicilliopsis_zonata', 'pseudomicrostroma_glucosiphilum', 'cryptococcus_wingfieldii', 'candida_oxycetoniae', 'emericellopsis_cladophorae', 'diutina_rugosa', 'sphaerulina_musiva', 'pseudozyma_flocculosa', 'trichophyton_verrucosum'}
Total #: 30 / 34


In [ ]:
# The other 4 not hit, where are they on the phylogenetic tree?

In [30]:
# Where are the hits? Some species are hit multiple times. Where are the hits?

In [ ]:
for seq, obj in target_seq_objects.items():
    print(seq, obj.attributes)
    print()

CACAAGCCCATCATGATTGC {'strands': ['R'], 'species': ['penicilliopsis_zonata'], 'genomic_locs': [19694895]}

TCATTACACCAAATTTCCAT {'strands': ['F', 'F'], 'species': ['candida_oxycetoniae', 'candida_pseudojiufengensis'], 'genomic_locs': [5540770, 373268]}

TCGGCGAGGAGGAACATCTC {'strands': ['F'], 'species': ['fusarium_oxysporum'], 'genomic_locs': [37384301]}

GGTGGTGAAATTAGAGATGA {'strands': ['R', 'R'], 'species': ['lodderomyces_elongisporus', 'naumovozyma_dairenensis'], 'genomic_locs': [9453659, 7404490]}

GCAATAATCTCCTGCACATT {'strands': ['F', 'R'], 'species': ['pseudogymnoascus_destructans', 'neurospora_crassa'], 'genomic_locs': [2320636, 14826864]}

ATTGGTACTGCTGGTGGTAC {'strands': ['F', 'F', 'R'], 'species': ['lodderomyces_elongisporus', 'candida_pseudojiufengensis', 'candida_pseudohaemulonii'], 'genomic_locs': [11885044, 9946384, 4272169]}

ATTGTCGAGAACCGTTTCAT {'strands': ['F', 'F', 'R'], 'species': ['dothidotthia_symphoricarpi', 'cladophialophora_carrionii', 'aspergillus_ibericus']

In [ ]:
# {
#    'daldinia_loculata': loc:[1, 2], strands:[F, R], guides:[A, B]
# }

class GuideStrandLoc:
    def __init__(self) -> None:
        self.strands: list[str] = list()
        self.guides: list[str] = list()
        self.genomic_locs: list[int] = list()


    @property
    def attributes(self) -> dict[str, list[str] | list[int]]:
        return dict({
            'strands': self.strands,
            'guides': self.guides,
            'genomic_locs': self.genomic_locs,
        })


    def update(self, guide: str, strand: str, loc: int) -> None:
        self.strands.append(strand)
        self.guides.append(guide)
        self.genomic_locs.append(loc)


species_to_info = dict()

for seq, obj in target_seq_objects.items():
    for i in range(len(obj.species)):
        species_to_info.setdefault(obj.species[i], GuideStrandLoc()).update(
            guide=seq,
            strand=obj.strands[i],
            loc=obj.genomic_locs[i]
        )


for k, v in species_to_info.items():
    print(k, v.attributes)

penicilliopsis_zonata {'strands': ['R', 'R', 'F', 'F'], 'guides': ['CACAAGCCCATCATGATTGC', 'ATGGTCTTGAGGAACTGCTT', 'AAGCCCTACTTCATCTACCC', 'GACAGCCACAGCAACATGTA'], 'genomic_locs': [19694895, 12320337, 7575293, 20245261]}
candida_oxycetoniae {'strands': ['F', 'F'], 'guides': ['TCATTACACCAAATTTCCAT', 'GGTTTTGAATGGAGAGAAGC'], 'genomic_locs': [5540770, 4988685]}
candida_pseudojiufengensis {'strands': ['F', 'F'], 'guides': ['TCATTACACCAAATTTCCAT', 'ATTGGTACTGCTGGTGGTAC'], 'genomic_locs': [373268, 9946384]}
fusarium_oxysporum {'strands': ['F'], 'guides': ['TCGGCGAGGAGGAACATCTC'], 'genomic_locs': [37384301]}
lodderomyces_elongisporus {'strands': ['R', 'F', 'R'], 'guides': ['GGTGGTGAAATTAGAGATGA', 'ATTGGTACTGCTGGTGGTAC', 'GCGTACAAGTGGTCGATACC'], 'genomic_locs': [9453659, 11885044, 6499363]}
naumovozyma_dairenensis {'strands': ['R'], 'guides': ['GGTGGTGAAATTAGAGATGA'], 'genomic_locs': [7404490]}
pseudogymnoascus_destructans {'strands': ['F', 'F', 'F'], 'guides': ['GCAATAATCTCCTGCACATT', 'AAGCCA